## Real estate investment in Milan

In [17]:
library(dplyr)

In [18]:
sale_raw<-read.csv2("result_ingatlan_sale.csv", as.is=T, sep=",")
rent_raw<-read.csv2("result_ingatlan_rent.csv", as.is=T, sep=",")

In [19]:
head(rent_raw)

,rooms,size,lon,price,adress1,adress2,lan,adress3,date,rentid
1,1,42,19.067871,110000,Boráros tér,Középső-Ferencváros – Malmok,47.48045,"9. kerület, Budapest",2016-07-11 11:28:21,21966456
2,4,116,19.054937,250000,Bajcsy-Zsilinszky út 5.,Belső-Terézváros,47.49846,"6. kerület, Budapest",2016-07-10 02:11:06,22453322
3,2 + 1,62,19.025272,220000,Toboz utca 2.,Mátyáshegy,47.53333,"3. kerület, Budapest",2016-07-14 00:39:07,22353414
4,3,71,19.03325,329438,Gellérthegy utca 8,Krisztinaváros,47.49581,"1. kerület, Budapest",2016-06-30 02:18:09,22015959
5,1,40,19.05703,100000,Dessewffy utca,Belső-Terézváros,47.50508,"6. kerület, Budapest",2016-07-14 14:30:36,22484130
6,2,52,19.188208,130000,Üllői út,Miklóstelep,47.439594,"18. kerület, Budapest",2016-07-13 14:24:39,22481131


In [20]:
head(sale_raw)

,adress2,lan,rentid,date,size,rooms,price,adress1,lon,adress3
1,Józsefváros – Corvinnegyed,47.48649,22277393,2016-04-22 17:28:13,75,3,49.9,Corvin köz,19.07121,"8. kerület, Budapest"
2,Külső-Terézváros,47.513428,22460943,2016-07-04 19:05:42,40,1,16.5,,19.066608,"6. kerület, Budapest"
3,Lipótváros,47.50794,22463469,2016-07-05 17:00:51,74,2 + 1,36.5,Honvéd utca,19.049728,"5. kerület, Budapest"
4,Albertfalva,47.44129,22455949,2016-07-01 15:00:59,45,1 + 1,23,Mezőkövesd út,19.03883,"11. kerület, Budapest"
5,Kurucles,47.52306,22328864,2016-05-11 11:46:26,48,2,22.56,Hárshegyi út 5-7,18.96904,"2. kerület, Budapest"
6,Kurucles,47.52306,22326046,2016-05-10 14:17:48,49,2,29.89,Hárshegyi út 5-7.,18.96904,"2. kerület, Budapest"


In [21]:
sale_raw$price<-as.numeric(sale_raw$price)*1000000

rent_raw$price_sqm<-rent_raw$price/rent_raw$size
sale_raw$price_sqm<-sale_raw$price/sale_raw$size

In [22]:
sale_raw$address<-sapply(sale_raw$adress3,function(x) head(unlist(strsplit(x, ",")),n=1)[1])
rent_raw$address<-sapply(rent_raw$adress3,function(x) head(unlist(strsplit(x, ",")),n=1)[1])

In [23]:
sale_clean<-sale_raw %>% 
            group_by(address,adress1, adress3, size, rooms) %>% 
            summarize(min_price_sqm=min(price_sqm),min_price=min(price),max_price_sqm=max(price_sqm),max_price=max(price),nr_ad=n())

rent_clean<-rent_raw %>% 
            group_by(address,adress1, adress3, size, rooms) %>% 
            summarize(min_price_sqm=min(price_sqm),min_price=min(price),max_price_sqm=max(price_sqm),max_price=max(price),nr_ad=n())

In [24]:
head(sale_clean)

,address,adress1,adress3,size,rooms,min_price_sqm,min_price,max_price_sqm,max_price,nr_ad
1,1. kerület,,"1. kerület, Budapest",18,1,697777.8,12560000,697777.8,12560000,1
2,1. kerület,,"1. kerület, Budapest",24,1,702083.3,16850000,702083.3,16850000,1
3,1. kerület,,"1. kerület, Budapest",26,1,688461.5,17900000,688461.5,17900000,3
4,1. kerület,,"1. kerület, Budapest",27,1,511111.1,13800000,674074.1,18200000,6
5,1. kerület,,"1. kerület, Budapest",28,1,607142.9,1.7e+07,607142.9,1.7e+07,1
6,1. kerület,,"1. kerület, Budapest",29,1,706896.6,20500000,758620.7,2.2e+07,4


In [25]:
x <- sale_clean$min_price_sqm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
sale_clean<-sale_clean[sale_clean$min_price_sqm > (qnt[1] - H),]
sale_clean<-sale_clean[sale_clean$min_price_sqm < (qnt[2] + H),]

x <- rent_clean$min_price_sqm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
rent_clean<-rent_clean[rent_clean$min_price_sqm > (qnt[1] - H),]
rent_clean<-rent_clean[rent_clean$min_price_sqm < (qnt[2] + H),]

In [26]:
sale_stat<-sale_clean %>%
           group_by(address) %>%
           summarize(sale_count=n(), avg_sale_price=mean(min_price_sqm),
                     sale_difficulty=mean(nr_ad)) %>%
           filter(sale_count>=20)

rent_stat<-rent_clean %>%
           group_by(address) %>%
           summarize(rent_count=n(), avg_rent_price=mean(min_price_sqm),
                     rent_difficulty=mean(nr_ad)) %>%
           filter(rent_count>=20)

compare<-inner_join(sale_stat,rent_stat, by=c("address"))

compare<-compare %>%
         mutate(rent_to_own_ratio=avg_sale_price/(avg_rent_price*12),sale_per_rent_count=sale_count/rent_count,
                gross_yield=avg_rent_price*12/avg_sale_price) %>%
         arrange(desc(gross_yield))

group_invest<- function(value){
    if (value<12.0) return("Vételre ajánlott")
    if (value>14.0) return("Bérlés ajánlott")
    return ("További értékelés szükséges")
}

        
rent_difficulty<- function(value){
    if (value<1.1) return("Könnyű")
    if (value>1.3) return("Nehéz")
    return ("Közepes")
}
        
 sale_difficulty<- function(value){
    if (value<2.05) return("Könnyű")
    if (value>2.3) return("Nehéz")
    return ("Közepes")
}
        
sale_group<- function(value){
    if (value<350000) return("Kedvező")
    if (value>480000) return("Drága")
    return ("Közepes")
}
        
rent_group<- function(value){
    if (value<2500) return("Kedvező")
    if (value>3000) return("Drága")
    return ("Közepes")
}
  
        
compare<-compare %>% 
         rowwise() %>% 
         mutate(investment= group_invest(rent_to_own_ratio), rent_difficulty_group=rent_difficulty(rent_difficulty),
                sale_difficulty_group=sale_difficulty(sale_difficulty), sale_price_group=sale_group(avg_sale_price),
                rent_price_group=rent_group(avg_rent_price))

compare %>%
        select(address,gross_yield,rent_to_own_ratio,rent_difficulty_group,sale_difficulty_group,
               rent_price_group,sale_price_group, investment) %>%
        arrange(address)

,address,gross_yield,rent_to_own_ratio,rent_difficulty_group,sale_difficulty_group,rent_price_group,sale_price_group,investment
1,1. kerület,0.07019063,14.24692,NehĂ©z,KĂ¶zepes,DrĂˇga,DrĂˇga,BĂ©rlĂ©s ajĂˇnlott
2,10. kerület,0.08932113,11.19556,KĂ¶nnyĹ±,KĂ¶zepes,KedvezĹ‘,KedvezĹ‘,VĂ©telre ajĂˇnlott
3,11. kerület,0.06681098,14.9676,KĂ¶zepes,KĂ¶zepes,KĂ¶zepes,DrĂˇga,BĂ©rlĂ©s ajĂˇnlott
4,12. kerület,0.06811785,14.68044,NehĂ©z,KĂ¶nnyĹ±,DrĂˇga,DrĂˇga,BĂ©rlĂ©s ajĂˇnlott
5,13. kerület,0.07581177,13.19056,KĂ¶zepes,NehĂ©z,DrĂˇga,DrĂˇga,TovĂˇbbi Ă©rtĂ©kelĂ©s szĂĽksĂ©ges
6,14. kerület,0.07366139,13.57563,KĂ¶zepes,NehĂ©z,KĂ¶zepes,KĂ¶zepes,TovĂˇbbi Ă©rtĂ©kelĂ©s szĂĽksĂ©ges
7,15. kerület,0.0944093,10.59218,KĂ¶nnyĹ±,NehĂ©z,KedvezĹ‘,KedvezĹ‘,VĂ©telre ajĂˇnlott
8,16. kerület,0.07613337,13.13485,KĂ¶nnyĹ±,KĂ¶nnyĹ±,KedvezĹ‘,KĂ¶zepes,TovĂˇbbi Ă©rtĂ©kelĂ©s szĂĽksĂ©ges
9,18. kerület,0.07577119,13.19763,KĂ¶nnyĹ±,NehĂ©z,KedvezĹ‘,KedvezĹ‘,TovĂˇbbi Ă©rtĂ©kelĂ©s szĂĽksĂ©ges
10,19. kerület,0.0976221,10.24358,KĂ¶nnyĹ±,KĂ¶nnyĹ±,KedvezĹ‘,KedvezĹ‘,VĂ©telre ajĂˇnlott


In [27]:
write.csv(compare, file="compare.csv", encoding="UTF-8")

ERROR: Error in write.table(compare, file = "compare.csv", encoding = "UTF-8", : unused argument (encoding = "UTF-8")
